# Applied Data Science Capstone:

## Identify New York Public High Schools with the Highest SAT Test Scores

### Table of contents
    Introduction: Business Problem
    Data
    Methodology
    Analysis
    Results and Discussion
    Conclusion

### Introduction: Business Problem 
The purpose of this project is to identifiy which public high school's within the five boroughs that make up the New York City area have the highest overall SAT scores, while isolating the public high school's that received the highest score for each section (math, reading, writing) of the SAT test and through k-mean clustering locate which borough has the highest performing public high schools. 

This report will be targeted to people who are looking to relocate or trying to apply to New York City area public high schools with the highest performing SAT scores. 


### Data 
Concentrating on the stated business problem, factors that will influence the location of high schoosl are:

The SAT score averages that have been provided by the College Board in each of the five boroughs creating the New York City area during the years 2014-2015.

The high school data that has been compiled and published by the New York City Department of Education.

Following data sources will be needed to extract/generate the required information:

Part 1: Preprocessing the average SAT 2014-2015 school year SAT test scores data from Kaggle. Crafting a datafram illustrating the avaerage SAT scores for every accredited high school in the New York City, five borough area. 

Part 2: Generate a map of the public high schools with the highest average total SAT test scores and three main sections of the test within the five boroughs area of New York City via folium.  

Part 3: Identify the closest venders to the public high schools with the highest SAT test scores with the 5 borough New York City area, via FourSquare API and geocoding.

## Part 1: Preprocessing real world data from Kaggle illustrating the avaerage SAT scores for every accredited high school in the New York City, five borough area. 

About this file:
New York City SAT test scores for the 2014-2015 school year.
borough: Common name for individual cities that togehter make up the New York City area.

Data set URL: https://www.kaggle.com/nycopendata/high-schools

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup # library for web scrapping  

#!conda install -c conda-forge geocoder --yes
import geocoder

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('done')

done


In [2]:
# Foursquare API 
CLIENT_ID = 'R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K' # your Foursquare ID
CLIENT_SECRET = '4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: R01LINGO2WC45KLRLKT3ZHU2QENAO2IPRK2N2ELOHRNK4P3K
CLIENT_SECRET:4JT1TWRMXMPLX5IOKNBAFU3L3ARXK4D5JJDPFK1CLRZM2ZVW


In [3]:
# Read in the data 
df = pd.read_csv("NY_SAT_Scores.csv")

In [4]:
# View the top rows of the dataset
df.head()

,School ID,School Name,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,...,End Time,Student Enrollment,Percent White,Percent Black,Percent Hispanic,Percent Asian,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Percent Tested
0,02M260,Clinton School Writers and Artists,Manhattan,M933,425 West 33rd Street,Manhattan,NY,10001,40.75321,-73.99786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,06M211,Inwood Early College for Health and Informatio...,Manhattan,M052,650 Academy Street,Manhattan,NY,10002,40.86605,-73.92486,...,3:00 PM,87.0,3.40%,21.80%,67.80%,4.60%,NaN,NaN,NaN,NaN
2,01M539,"New Explorations into Science, Technology and ...",Manhattan,M022,111 Columbia Street,Manhattan,NY,10002,40.71873,-73.97943,...,4:00 PM,1735.0,28.60%,13.30%,18.00%,38.50%,657.0,601.0,601.0,91.00%
3,02M294,Essex Street Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,...,2:45 PM,358.0,11.70%,38.50%,41.30%,5.90%,395.0,411.0,387.0,78.90%
4,02M308,Lower Manhattan Arts Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,...,3:00 PM,383.0,3.10%,28.20%,56.90%,8.60%,418.0,428.0,415.0,65.10%


In [5]:
# Dropping columns that are not relevant to the problem
df.drop(['Start Time','End Time','Phone Number','Percent White','Percent Black','Percent Hispanic','Percent Asian','Percent Tested'], axis = 1, inplace = True)
df.head()

,School ID,School Name,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing)
0,02M260,Clinton School Writers and Artists,Manhattan,M933,425 West 33rd Street,Manhattan,NY,10001,40.75321,-73.99786,NaN,NaN,NaN,NaN
1,06M211,Inwood Early College for Health and Informatio...,Manhattan,M052,650 Academy Street,Manhattan,NY,10002,40.86605,-73.92486,87.0,NaN,NaN,NaN
2,01M539,"New Explorations into Science, Technology and ...",Manhattan,M022,111 Columbia Street,Manhattan,NY,10002,40.71873,-73.97943,1735.0,657.0,601.0,601.0
3,02M294,Essex Street Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,358.0,395.0,411.0,387.0
4,02M308,Lower Manhattan Arts Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,383.0,418.0,428.0,415.0


In [6]:
# re-name column

df.rename(columns={'School Name':'School'}, inplace=True)

In [7]:
# insert column with average total SAT scores

df.insert(14,"Average Score (SAT Total)","0", allow_duplicates=False)

df['Average Score (SAT Total)']=df.iloc[:,-4:].sum(axis=1)

In [8]:
df.head()

,School ID,School,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Average Score (SAT Total)
0,02M260,Clinton School Writers and Artists,Manhattan,M933,425 West 33rd Street,Manhattan,NY,10001,40.75321,-73.99786,NaN,NaN,NaN,NaN,0.0
1,06M211,Inwood Early College for Health and Informatio...,Manhattan,M052,650 Academy Street,Manhattan,NY,10002,40.86605,-73.92486,87.0,NaN,NaN,NaN,0.0
2,01M539,"New Explorations into Science, Technology and ...",Manhattan,M022,111 Columbia Street,Manhattan,NY,10002,40.71873,-73.97943,1735.0,657.0,601.0,601.0,1859.0
3,02M294,Essex Street Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,358.0,395.0,411.0,387.0,1193.0
4,02M308,Lower Manhattan Arts Academy,Manhattan,M445,350 Grand Street,Manhattan,NY,10002,40.71687,-73.98953,383.0,418.0,428.0,415.0,1261.0


## Methodology

#### The methodology in this project consists of two sections, exploratory data analysis and machine learning modeling.

#### Exploratory Data Analysis: 
Visualize the areas of the five borough that posses the highest public high schools with the average total SAT test scores. Idenify boroughs that have the highest public high schools with within each section of the SAT test.

#### Modelling: 
To assist families/students locate public high schools with the higest average SAT test scores. Then identifing the closest vendors to the public high schools FourSquare data. The vendor clustering will enable families/students to be aware of the closest local venders for further evaluation of not only the public high school, but surrounding area as well. 

## Part II -  Generate a map of the public high schools with the highest average total SAT test scores and three main sections of the test within the five boroughs area of New York City via folium.  

In [9]:
# import libraries

# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

Matplotlib version:  3.1.1


In [10]:
# Matplotlib and associated plotting modules
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

import requests
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans

print('done')

done


In [11]:
place ='New York City, New York'

geolocator=Nominatim(user_agent='can_explorer')
location = geolocator.geocode(place)
latitude=location.latitude
longitude=location.longitude
print('The coordinates of New York City are {}, {}.'.format(latitude, longitude))

The coordinates of New York City are 40.7127281, -74.0060152.


In [12]:
## Sort the highest average test scores in each section to view the location within the 5 bouroughs

dftotal= df.sort_values(['Average Score (SAT Total)'], ascending = False, axis = 0, inplace = True )

dftop10total= df.sort_values(['Average Score (SAT Total)'], ascending = False, axis = 0, inplace = True )
dftop10total = df.head(10) 
dftop10total 

,School ID,School,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Average Score (SAT Total)
105,02M475,Stuyvesant High School,Manhattan,M477,345 Chambers Street,Manhattan,NY,10282,40.71775,-74.01405,3296.0,754.0,697.0,693.0,2144.0
110,31R605,Staten Island Technical High School,Staten Island,R440,485 Clawson Street,Staten Island,NY,10306,40.56791,-74.11536,1247.0,711.0,660.0,670.0,2041.0
203,10X445,Bronx High School of Science,Bronx,X445,75 West 205th Street,Bronx,NY,10468,40.87706,-73.88978,3015.0,714.0,660.0,667.0,2041.0
208,10X696,High School of American Studies at Lehman College,Bronx,X905,2925 Goulden Avenue,Bronx,NY,10468,40.87126,-73.89752,376.0,669.0,672.0,672.0,2013.0
385,25Q525,Townsend Harris High School,Queens,Q515,149-11 Melbourne Avenue,Flushing,NY,11367,40.73441,-73.82142,1132.0,680.0,640.0,661.0,1981.0
424,28Q687,Queens High School for the Sciences at York Co...,Queens,Q774,94-50 159th Street,Jamaica,NY,11433,40.70100,-73.79815,426.0,701.0,621.0,625.0,1947.0
7,01M696,Bard High School Early College,Manhattan,M097,525 East Houston Street,Manhattan,NY,10002,40.71896,-73.97607,545.0,634.0,641.0,639.0,1914.0
326,13K430,Brooklyn Technical High School,Brooklyn,K430,29 Ft Greene Place,Brooklyn,NY,11217,40.68811,-73.97675,5447.0,682.0,608.0,606.0,1896.0
55,02M416,Eleanor Roosevelt High School,Manhattan,M855,411 East 76th Street,Manhattan,NY,10021,40.77012,-73.95338,541.0,641.0,617.0,631.0,1889.0
80,05M692,"High School for Mathematics, Science, and Engi...",Manhattan,M812,240 Convent Avenue,Manhattan,NY,10031,40.82112,-73.94885,470.0,683.0,610.0,596.0,1889.0


In [13]:
NYMapTotal = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, school in zip(dftop10total['Latitude'], dftop10total['Longitude'], dftop10total['Borough'], dftop10total['School']):
    label = '{}, {}'.format(borough, school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NYMapTotal)  
    
NYMapTotal

In [14]:
## Sort the highest average test scores in each section to view the location within the 5 bouroughs

dftop10Math= df.sort_values(['Average Score (SAT Math)'], ascending = False, axis = 0, inplace = True )

dftop10Math = df.head(10) 
dftop10Math

,School ID,School,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Average Score (SAT Total)
105,02M475,Stuyvesant High School,Manhattan,M477,345 Chambers Street,Manhattan,NY,10282,40.71775,-74.01405,3296.0,754.0,697.0,693.0,2144.0
203,10X445,Bronx High School of Science,Bronx,X445,75 West 205th Street,Bronx,NY,10468,40.87706,-73.88978,3015.0,714.0,660.0,667.0,2041.0
110,31R605,Staten Island Technical High School,Staten Island,R440,485 Clawson Street,Staten Island,NY,10306,40.56791,-74.11536,1247.0,711.0,660.0,670.0,2041.0
424,28Q687,Queens High School for the Sciences at York Co...,Queens,Q774,94-50 159th Street,Jamaica,NY,11433,40.70100,-73.79815,426.0,701.0,621.0,625.0,1947.0
80,05M692,"High School for Mathematics, Science, and Engi...",Manhattan,M812,240 Convent Avenue,Manhattan,NY,10031,40.82112,-73.94885,470.0,683.0,610.0,596.0,1889.0
326,13K430,Brooklyn Technical High School,Brooklyn,K430,29 Ft Greene Place,Brooklyn,NY,11217,40.68811,-73.97675,5447.0,682.0,608.0,606.0,1896.0
385,25Q525,Townsend Harris High School,Queens,Q515,149-11 Melbourne Avenue,Flushing,NY,11367,40.73441,-73.82142,1132.0,680.0,640.0,661.0,1981.0
208,10X696,High School of American Studies at Lehman College,Bronx,X905,2925 Goulden Avenue,Bronx,NY,10468,40.87126,-73.89752,376.0,669.0,672.0,672.0,2013.0
2,01M539,"New Explorations into Science, Technology and ...",Manhattan,M022,111 Columbia Street,Manhattan,NY,10002,40.71873,-73.97943,1735.0,657.0,601.0,601.0,1859.0
55,02M416,Eleanor Roosevelt High School,Manhattan,M855,411 East 76th Street,Manhattan,NY,10021,40.77012,-73.95338,541.0,641.0,617.0,631.0,1889.0


In [15]:
NYmapMath = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, school in zip(dftop10Math['Latitude'], dftop10Math['Longitude'], dftop10Math['Borough'], dftop10Math['School']):
    label = '{}, {}'.format(borough, school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NYmapMath)  
    
NYmapMath

In [16]:
## Sort the highest average test scores in each section to view the location within the 5 bouroughs

dftop10Reading= df.sort_values(['Average Score (SAT Reading)'], ascending = False, axis = 0, inplace = True )

dftop10Reading = df.head(10) 
dftop10Reading

,School ID,School,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Average Score (SAT Total)
105,02M475,Stuyvesant High School,Manhattan,M477,345 Chambers Street,Manhattan,NY,10282,40.71775,-74.01405,3296.0,754.0,697.0,693.0,2144.0
208,10X696,High School of American Studies at Lehman College,Bronx,X905,2925 Goulden Avenue,Bronx,NY,10468,40.87126,-73.89752,376.0,669.0,672.0,672.0,2013.0
110,31R605,Staten Island Technical High School,Staten Island,R440,485 Clawson Street,Staten Island,NY,10306,40.56791,-74.11536,1247.0,711.0,660.0,670.0,2041.0
203,10X445,Bronx High School of Science,Bronx,X445,75 West 205th Street,Bronx,NY,10468,40.87706,-73.88978,3015.0,714.0,660.0,667.0,2041.0
7,01M696,Bard High School Early College,Manhattan,M097,525 East Houston Street,Manhattan,NY,10002,40.71896,-73.97607,545.0,634.0,641.0,639.0,1914.0
385,25Q525,Townsend Harris High School,Queens,Q515,149-11 Melbourne Avenue,Flushing,NY,11367,40.73441,-73.82142,1132.0,680.0,640.0,661.0,1981.0
424,28Q687,Queens High School for the Sciences at York Co...,Queens,Q774,94-50 159th Street,Jamaica,NY,11433,40.70100,-73.79815,426.0,701.0,621.0,625.0,1947.0
251,30Q580,Baccalaureate School for Global Education,Queens,Q798,34-12 36th Avenue,Astoria,NY,11106,40.75517,-73.92682,478.0,633.0,620.0,628.0,1881.0
55,02M416,Eleanor Roosevelt High School,Manhattan,M855,411 East 76th Street,Manhattan,NY,10021,40.77012,-73.95338,541.0,641.0,617.0,631.0,1889.0
80,05M692,"High School for Mathematics, Science, and Engi...",Manhattan,M812,240 Convent Avenue,Manhattan,NY,10031,40.82112,-73.94885,470.0,683.0,610.0,596.0,1889.0


In [17]:
NYMapReading = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, school in zip(dftop10Reading['Latitude'], dftop10Reading['Longitude'], dftop10Reading['Borough'], dftop10Reading['School']):
    label = '{}, {}'.format(borough, school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NYMapReading)  
    
NYMapReading

In [18]:
## Sort the highest average test scores in each section to view the location within the 5 bouroughs

dftop10Writing= df.sort_values(['Average Score (SAT Writing)'], ascending = False, axis = 0, inplace = True )

dftop10Writing = df.head(10) 
dftop10Writing

,School ID,School,Borough,Building Code,Street Address,City,State,Zip Code,Latitude,Longitude,Student Enrollment,Average Score (SAT Math),Average Score (SAT Reading),Average Score (SAT Writing),Average Score (SAT Total)
105,02M475,Stuyvesant High School,Manhattan,M477,345 Chambers Street,Manhattan,NY,10282,40.71775,-74.01405,3296.0,754.0,697.0,693.0,2144.0
208,10X696,High School of American Studies at Lehman College,Bronx,X905,2925 Goulden Avenue,Bronx,NY,10468,40.87126,-73.89752,376.0,669.0,672.0,672.0,2013.0
110,31R605,Staten Island Technical High School,Staten Island,R440,485 Clawson Street,Staten Island,NY,10306,40.56791,-74.11536,1247.0,711.0,660.0,670.0,2041.0
203,10X445,Bronx High School of Science,Bronx,X445,75 West 205th Street,Bronx,NY,10468,40.87706,-73.88978,3015.0,714.0,660.0,667.0,2041.0
385,25Q525,Townsend Harris High School,Queens,Q515,149-11 Melbourne Avenue,Flushing,NY,11367,40.73441,-73.82142,1132.0,680.0,640.0,661.0,1981.0
7,01M696,Bard High School Early College,Manhattan,M097,525 East Houston Street,Manhattan,NY,10002,40.71896,-73.97607,545.0,634.0,641.0,639.0,1914.0
55,02M416,Eleanor Roosevelt High School,Manhattan,M855,411 East 76th Street,Manhattan,NY,10021,40.77012,-73.95338,541.0,641.0,617.0,631.0,1889.0
251,30Q580,Baccalaureate School for Global Education,Queens,Q798,34-12 36th Avenue,Astoria,NY,11106,40.75517,-73.92682,478.0,633.0,620.0,628.0,1881.0
424,28Q687,Queens High School for the Sciences at York Co...,Queens,Q774,94-50 159th Street,Jamaica,NY,11433,40.70100,-73.79815,426.0,701.0,621.0,625.0,1947.0
241,24Q299,Bard High School Early College Queens,Queens,Q735,30-20 Thomson Avenue,Long Island City,NY,11101,40.74509,-73.93681,623.0,631.0,598.0,610.0,1839.0


In [19]:
NYmapWriting = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, school in zip(dftop10Writing['Latitude'], dftop10Writing['Longitude'], dftop10Writing['Borough'], dftop10Writing['School']):
    label = '{}, {}'.format(borough,school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(NYmapWriting)  
    
NYmapWriting

### Part III: Identify the closest venders to the public high schools with the highest SAT test scores with the 5 borough New York City area, via FourSquare API and geocoding.

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['School', 
                  'School Latitude', 
                  'School Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
NYvenues = getNearbyVenues(names=dftop10total['School'],
                                   latitudes=dftop10total['Latitude'],
                                   longitudes=dftop10total['Longitude'],
                                  )

Stuyvesant High School
Staten Island Technical High School
Bronx High School of Science
High School of American Studies at Lehman College
Townsend Harris High School
Queens High School for the Sciences at York College
Bard High School Early College
Brooklyn Technical High School
Eleanor Roosevelt High School
High School for Mathematics, Science, and Engineering at City College


In [22]:
NYvenues.head()

,School,School Latitude,School Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Stuyvesant High School,40.71775,-74.01405,Teardrop Park,40.716843,-74.015576,Park
1,Stuyvesant High School,40.71775,-74.01405,North End Ave Dog Run,40.716876,-74.016149,Dog Run
2,Stuyvesant High School,40.71775,-74.01405,Asphalt Green Battery Park City,40.715924,-74.014673,Athletics & Sports
3,Stuyvesant High School,40.71775,-74.01405,SoulCycle TriBeCa,40.716178,-74.013007,Cycle Studio
4,Stuyvesant High School,40.71775,-74.01405,Nelson A. Rockefeller Park,40.717095,-74.016716,Park


In [23]:
# one hot encoding
schoolonehot = pd.get_dummies(NYvenues[['Venue']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
schoolonehot['School'] = NYvenues['School'] 

fixed_columns = [schoolonehot.columns[-1]] + list(schoolonehot.columns[:-1])
schoolonehot = schoolonehot[fixed_columns]

schoolonehot.head()

,School,150th And Jewel Bus Stop,2nd Ave Deli,2nd Floor Bar & Essen,APEX - Lehman College,Agata & Valentina UES Market,Anita's Roti Shop,Applebee's Grill + Bar,Art Gallery @Lehman College,Asphalt Green Battery Park City,BAM Fisher,BAM Harvey Theater,BAM Howard Gilman Opera House,BAM Rose Cinemas,BAMcafé,BRIC House,Bakhtar Halal Kabab Main Street,Bareburger,Barnes & Noble,Battery Park City Ballfields,Bedford Cafe Restaurant,Big G Deli,Black Forest Brooklyn,Blaze Pizza,Blink Fitness,Bookstore - Lehman College,Boqueria,Brahman Yoga Studio,Brian Watkins Tennis Center,Brooklyn Academy of Music (BAM),Brother's Pizza,CTown Supermarkets,CUNY Queens College Track,CVS pharmacy,Cafe One,Cafe Paulette,Campagnola Restaurant,Carlos & Gabby's,Carman Cafe,Carvel,Cedar Grove Playground,Cesar's Empanadas,Chase Bank,Chinelos,Chipotle Mexican Grill,Choi Yuan - Chinese Restaurant,Cigar Inn,Citi Bike Station,Clove Indian Restaurant & Bar,Coffee Project NY,Conrad New York,Cơm Tấm Ninh Kiều,Dollar Tree,Don Nico's,Double Dutch Espresso,Duane Reade,Dunkin',E 204 St & Grand Concourse,East River Park,East River Park Fields,East River Park Track,East River Running Path,Eastside Massage Therapy,Empire East,Filtered Coffee,Fine Fare Supermarket,Firestone Complete Auto Care,Fitzgerald Gym,Five Below,Five Guys,Fort Greene Park Greenmarket,Fort Greene Park Tennis Courts,Frijolito's,Fumo,GNC,GameStop,Geisha Japanese Cuisine,Godwin-Ternbach Museum - Queens College,Golden Krust Caribbean Restaurant,Grand Banks,Greene Grape Annex,Greenlight Bookstore,H&M,Habana Outpost,Hamilton Grange National Memorial,Harlem Cigar Room,Harlem School of the Arts,Heidi's House By The Side Of The Road,Hogshead Tavern,HomeGoods,House of Jai Yoga,Hylan Plaza,Imagine Swimming - Tribeca Flagship,Inès,Irish Hunger Memorial,Jamaica Center,Jamaica Market,Jamaica Multiplex Cinemas,Jamaica Performing Arts Center (JPAC),Jerome Food Center Corp.,Jerome Park Reservoir,Jerome's Pizza,Jersey Mike's Subs,Jones Wood Foundry,K&G,Kennedy Fried Chicken,King Gourmet Deli (Fancy Deli),Kingsbridge Coffee Shop & Deli,LJ Pizza,La Nueva Flor de Broadway,Las Maravillas De Mexico,Laughing Man Coffee & Tea,Le Moulin A Cafe,Lehman Center For The Performing Arts,Lehman College Ballfield,Little Caesars Pizza,Little Ceasars,"MTA MaBSTOA Bus at W. Kingsbridge Road & University Ave (MLK Blvd): (Bx3, Bx9, Bx32)",MTA Subway - Bedford Park Blvd (B/D),Madden's Pub,Manhattanville Coffee,Manny deli,Margherita Pizza,Mark Morris Dance Center,Mashbill Wine & Spirits,Melt Massage,Modell's Sporting Goods,Monte Carlo,Montezuma Mexican Restaurant,Morton Williams Supermarket,Mr. Mango,Muse Paintbar,NR,Naomi's Kosher Pizza,Napoli Pizza,National Restaurant & Coffee,Nelson A. Rockefeller Park,New Capital Restaurant,New China Buffet,North End Ave Dog Run,Old Navy,Olga's Pizza,Orwasher's Bakery,Oslo Coffee Roasters,Oso,P.S. 499,PP Boy Japan Teriyaki & Sushi,Panda Express,Peaches Hothouse,Pequeña,Perista,Petit Paulette,Pier 25 - Hudson River Park,Pier 25 Beach Volleyball,Pier 25 Mini Golf,Pier 25 Playground,Pier 26 - Hudson River Park,Pilates Reforming New York - E. 76th St,Polarn O. Pyret,Popeyes Louisiana Kitchen,Queen Sheeba,Queens College Tennis Center,Queens Pita Bakery,R.O.K.C,Regal Battery Park,Reservoir,Retro Fitness,S & M Pharmacy Inc,SIPS + MAKER,Saffron,Salvo's Pizzabar,Sashimi Express,Season's,Sedutto,Shake Shack,Shalon Restaurant,Shopper's World,Something Sweet,SoulCycle TriBeCa,St. Nicholas Park,St. Nick's Dog Run,Starbucks,Stuyvesant High School,Subway,Subway® Restaurants,Sugar Hill Cafe,Super Stop & Shop,Sushi Ishikawa,Sushi Sushi,Sushi Tokyo,Sushi of Gari,Sweet Chef Southern Style Bakery,T-Mobile,THEP Thai Restaurant,Taco Bell / Pizza Hut,Tanoshi Sushi,Teardrop Park,Terroir,The Center For Fiction,The Children's Place,The Edge Harlem,The Grange Bar & Eatery,The Greene Grape Wine & Spirits,The Harlem Scrabble Room,The Meatball Shop,The Palm,The Pony Bar,The Rockwell Place,The Shala,The Sweet Shop NYC,The UPS Store,Theatr

In [24]:
school_grouped = schoolonehot.groupby('School').mean().reset_index()
school_grouped

,School,150th And Jewel Bus Stop,2nd Ave Deli,2nd Floor Bar & Essen,APEX - Lehman College,Agata & Valentina UES Market,Anita's Roti Shop,Applebee's Grill + Bar,Art Gallery @Lehman College,Asphalt Green Battery Park City,BAM Fisher,BAM Harvey Theater,BAM Howard Gilman Opera House,BAM Rose Cinemas,BAMcafé,BRIC House,Bakhtar Halal Kabab Main Street,Bareburger,Barnes & Noble,Battery Park City Ballfields,Bedford Cafe Restaurant,Big G Deli,Black Forest Brooklyn,Blaze Pizza,Blink Fitness,Bookstore - Lehman College,Boqueria,Brahman Yoga Studio,Brian Watkins Tennis Center,Brooklyn Academy of Music (BAM),Brother's Pizza,CTown Supermarkets,CUNY Queens College Track,CVS pharmacy,Cafe One,Cafe Paulette,Campagnola Restaurant,Carlos & Gabby's,Carman Cafe,Carvel,Cedar Grove Playground,Cesar's Empanadas,Chase Bank,Chinelos,Chipotle Mexican Grill,Choi Yuan - Chinese Restaurant,Cigar Inn,Citi Bike Station,Clove Indian Restaurant & Bar,Coffee Project NY,Conrad New York,Cơm Tấm Ninh Kiều,Dollar Tree,Don Nico's,Double Dutch Espresso,Duane Reade,Dunkin',E 204 St & Grand Concourse,East River Park,East River Park Fields,East River Park Track,East River Running Path,Eastside Massage Therapy,Empire East,Filtered Coffee,Fine Fare Supermarket,Firestone Complete Auto Care,Fitzgerald Gym,Five Below,Five Guys,Fort Greene Park Greenmarket,Fort Greene Park Tennis Courts,Frijolito's,Fumo,GNC,GameStop,Geisha Japanese Cuisine,Godwin-Ternbach Museum - Queens College,Golden Krust Caribbean Restaurant,Grand Banks,Greene Grape Annex,Greenlight Bookstore,H&M,Habana Outpost,Hamilton Grange National Memorial,Harlem Cigar Room,Harlem School of the Arts,Heidi's House By The Side Of The Road,Hogshead Tavern,HomeGoods,House of Jai Yoga,Hylan Plaza,Imagine Swimming - Tribeca Flagship,Inès,Irish Hunger Memorial,Jamaica Center,Jamaica Market,Jamaica Multiplex Cinemas,Jamaica Performing Arts Center (JPAC),Jerome Food Center Corp.,Jerome Park Reservoir,Jerome's Pizza,Jersey Mike's Subs,Jones Wood Foundry,K&G,Kennedy Fried Chicken,King Gourmet Deli (Fancy Deli),Kingsbridge Coffee Shop & Deli,LJ Pizza,La Nueva Flor de Broadway,Las Maravillas De Mexico,Laughing Man Coffee & Tea,Le Moulin A Cafe,Lehman Center For The Performing Arts,Lehman College Ballfield,Little Caesars Pizza,Little Ceasars,"MTA MaBSTOA Bus at W. Kingsbridge Road & University Ave (MLK Blvd): (Bx3, Bx9, Bx32)",MTA Subway - Bedford Park Blvd (B/D),Madden's Pub,Manhattanville Coffee,Manny deli,Margherita Pizza,Mark Morris Dance Center,Mashbill Wine & Spirits,Melt Massage,Modell's Sporting Goods,Monte Carlo,Montezuma Mexican Restaurant,Morton Williams Supermarket,Mr. Mango,Muse Paintbar,NR,Naomi's Kosher Pizza,Napoli Pizza,National Restaurant & Coffee,Nelson A. Rockefeller Park,New Capital Restaurant,New China Buffet,North End Ave Dog Run,Old Navy,Olga's Pizza,Orwasher's Bakery,Oslo Coffee Roasters,Oso,P.S. 499,PP Boy Japan Teriyaki & Sushi,Panda Express,Peaches Hothouse,Pequeña,Perista,Petit Paulette,Pier 25 - Hudson River Park,Pier 25 Beach Volleyball,Pier 25 Mini Golf,Pier 25 Playground,Pier 26 - Hudson River Park,Pilates Reforming New York - E. 76th St,Polarn O. Pyret,Popeyes Louisiana Kitchen,Queen Sheeba,Queens College Tennis Center,Queens Pita Bakery,R.O.K.C,Regal Battery Park,Reservoir,Retro Fitness,S & M Pharmacy Inc,SIPS + MAKER,Saffron,Salvo's Pizzabar,Sashimi Express,Season's,Sedutto,Shake Shack,Shalon Restaurant,Shopper's World,Something Sweet,SoulCycle TriBeCa,St. Nicholas Park,St. Nick's Dog Run,Starbucks,Stuyvesant High School,Subway,Subway® Restaurants,Sugar Hill Cafe,Super Stop & Shop,Sushi Ishikawa,Sushi Sushi,Sushi Tokyo,Sushi of Gari,Sweet Chef Southern Style Bakery,T-Mobile,THEP Thai Restaurant,Taco Bell / Pizza Hut,Tanoshi Sushi,Teardrop Park,Terroir,The Center For Fiction,The Children's Place,The Edge Harlem,The Grange Bar & Eatery,The Greene Grape Wine & Spirits,The Harlem Scrabble Room,The Meatball Shop,The Palm,The Pony Bar,The Rockwell Place,The Shala,The Sweet Shop NYC,The UPS Store,Theatr

In [25]:
close_venues = 5

for stuff in school_grouped['School']:
    print("----"+stuff+"----")
    temp = school_grouped[school_grouped['School'] == stuff].T.reset_index()
    temp.columns = ['venue','dist']
    temp = temp.iloc[1:]
    temp['dist'] = temp['dist'].astype(float)
    temp = temp.round({'dist': 2})
    print(temp.sort_values('dist', ascending=False).reset_index(drop=True).head(close_venues))
    print('\n')

----Bard High School Early College----
                         venue  dist
0        East River Park Track  0.17
1  Brian Watkins Tennis Center  0.17
2       East River Park Fields  0.17
3              East River Park  0.17
4      East River Running Path  0.17


----Bronx High School of Science----
                     venue  dist
0  Bedford Cafe Restaurant  0.06
1          W. 205th Street  0.06
2            Blink Fitness  0.06
3    Kennedy Fried Chicken  0.06
4               Big G Deli  0.06


----Brooklyn Technical High School----
                   venue  dist
0           Melt Massage  0.03
1          Cafe Paulette  0.03
2         Habana Outpost  0.03
3  Black Forest Brooklyn  0.03
4             UrbanGlass  0.03


----Eleanor Roosevelt High School----
                      venue  dist
0  Eastside Massage Therapy  0.03
1             Tanoshi Sushi  0.03
2        The Sweet Shop NYC  0.03
3         Orwasher's Bakery  0.03
4      Oslo Coffee Roasters  0.03


----High School for Mathemati

## Results and Discussion

#### Results:
Stuyvesant High School located in Manhatten was the top performing average SAT test score in all sections of the SAT test. The same ten high schools were listed for each section of test and subsequently, had the highest total average test scores as well. Those schools are:Stuyvesant High School, Staten Island Technical High School, Bronx High School of Science, High School of American Studies at Lehman College, Townsend Harris High School, Queens High School for the Sciences at York College, Bard High School Early College, Brooklyn Technical High School, Eleanor Roosevelt High School, High School for Mathematics, Science, and Engineering at City College.


#### Discussion:
The aim of this project is to help families and students identify public high schools within the New York City, five boroughs area with the highest average SAT test scores. xpats can chose the neighborhoods to which they want to relocate based on the most common SAT test scores are used help colleges interpret students' overall academic performance in relation to the national applicant pool. For families and students that are planning for college admission after high school, the public high schools with the total highest test scores are of interest. Additionlly, if a student is weak in one section of the test, a public high school that has high average test scores within this academic disapline is of intersest as well.  Lastly, identifing the closest local vendors to the highest total average test score public high school assisst family with planning for after school activities, pick-up/drop-off locations, and an overall general idea of what is within walking distance of the school. 



## Conclusion:
This project aids families and students alike with a better understanding of public high schools with high academic achievements with respect to the SAT test scores. Venues near the public high schools aslo allow families and students to find local, walking distance places near the public high schools for planning purposes and neighborhood associations. Reconizing that this project was singularly focused on SAT test scores, future projects could use this data as a platform for more indepth analysis of public high schools located within the New York City, five boroughs area. 